In [642]:
pip install pyreadstat requests scikit-learn numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [643]:
import numpy as np
import pandas as pd
import pyreadstat
import requests
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score#, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [644]:
# Dataframe for the demographics data

url_demo ='https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.xpt'

file_name_demo = "DEMO_L.xpt"

response = requests.get(url_demo)
if response.status_code == 200:
    with open(file_name_demo, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

demo, meta_demo = pyreadstat.read_xport(file_name_demo, encoding='latin1')
print(demo.info())
print(demo.head())

File downloaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      11933 non-null  float64
 1   SDDSRVYR  11933 non-null  float64
 2   RIDSTATR  11933 non-null  float64
 3   RIAGENDR  11933 non-null  float64
 4   RIDAGEYR  11933 non-null  float64
 5   RIDAGEMN  377 non-null    float64
 6   RIDRETH1  11933 non-null  float64
 7   RIDRETH3  11933 non-null  float64
 8   RIDEXMON  8860 non-null   float64
 9   RIDEXAGM  2787 non-null   float64
 10  DMQMILIZ  8301 non-null   float64
 11  DMDBORN4  11914 non-null  float64
 12  DMDYRUSR  1875 non-null   float64
 13  DMDEDUC2  7794 non-null   float64
 14  DMDMARTZ  7792 non-null   float64
 15  RIDEXPRG  1503 non-null   float64
 16  DMDHHSIZ  11933 non-null  float64
 17  DMDHRGND  4115 non-null   float64
 18  DMDHRAGZ  4124 non-null   float64
 19  DMDHREDZ  3746 non-null   float64
 20

In [645]:
demo_cols = {
'SEQN':'id',
'SDDSRVYR' : 'Data release cycle',
'RIDSTATR' : 'Interview/Examination status',
'RIAGENDR' : 'Gender',
'RIDAGEYR' : 'Age in years at screening',
'RIDAGEMN' : 'Age in months at screening - 0 to 24 mos',
'RIDRETH1' : 'Race/Hispanic origin',
'RIDRETH3' : 'Race/Hispanic origin w/ NH Asian',
'RIDEXMON' : 'Six-month time period',
'RIDEXAGM' : 'Age in months at exam - 0 to 19 years',
'DMQMILIZ' : 'Served active duty in US Armed Forces',
'DMDBORN4' : 'Country of birth',
'DMDYRUSR' : 'Length of time in US',
'DMDEDUC2' : 'Education level - Adults 20+',
'DMDMARTZ' : 'Marital status',
'RIDEXPRG' : 'Pregnancy status at exam',
'DMDHHSIZ' : 'Total number of people in the Household',
'DMDHRGND' : 'HH ref persons gender',
'DMDHRAGZ' : 'HH ref persons age in years',
'DMDHREDZ': 'HH ref persons education level',
'DMDHRMAZ' : 'HH ref persons marital status',
'DMDHSEDZ' : 'HH ref persons spouses education level',
'WTINT2YR' : 'Full sample 2-year interview weight',
'WTMEC2YR' : 'Full sample 2-year MEC exam weight',
'SDMVSTRA' : 'Masked variance pseudo-stratum',
'SDMVPSU' : 'Masked variance pseudo-PSU',
'INDFMPIR' : 'Ratio of family income to poverty'
}

demo = demo.rename(columns=demo_cols)
demo.head()

,id,Data release cycle,Interview/Examination status,Gender,Age in years at screening,Age in months at screening - 0 to 24 mos,Race/Hispanic origin,Race/Hispanic origin w/ NH Asian,Six-month time period,Age in months at exam - 0 to 19 years,...,HH ref persons gender,HH ref persons age in years,HH ref persons education level,HH ref persons marital status,HH ref persons spouses education level,Full sample 2-year interview weight,Full sample 2-year MEC exam weight,Masked variance pseudo-stratum,Masked variance pseudo-PSU,Ratio of family income to poverty
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60


In [646]:
demo['id'].nunique()

11933

In [647]:
# Filter demo for 20 and over

adult_demo = demo[demo['Age in years at screening'] >= 20.0]

adult_demo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7809 entries, 0 to 11932
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        7809 non-null   float64
 1   Data release cycle                        7809 non-null   float64
 2   Interview/Examination status              7809 non-null   float64
 3   Gender                                    7809 non-null   float64
 4   Age in years at screening                 7809 non-null   float64
 5   Age in months at screening - 0 to 24 mos  0 non-null      float64
 6   Race/Hispanic origin                      7809 non-null   float64
 7   Race/Hispanic origin w/ NH Asian          7809 non-null   float64
 8   Six-month time period                     6064 non-null   float64
 9   Age in months at exam - 0 to 19 years     0 non-null      float64
 10  Served active duty in US Armed Forces   

In [648]:
# Sleep 
"""
SEQN - Respondent sequence number
SLQ300 - Usual sleep time on weekdays or workdays
SLQ310 - Usual wake time on weekdays or workdays
SLD012 - Sleep hours - weekdays or workdays
SLQ320 - Usual sleep time on weekends
SLQ330 - Usual wake time on weekends
SLD013 - Sleep hours - weekends 
"""

url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/SLQ_L.xpt'

file_name = "SLQ_L.xpt"

response = requests.get(url)
if response.status_code == 200:
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

sleep, meta_sleep = pyreadstat.read_xport(file_name, encoding='latin1')

sleep_cols = {
    'SEQN':'id',
    "SLQ300": "Usual sleep time on weekdays or workdays",
    "SLQ310": "Usual wake time on weekdays or workdays",
    "SLD012": "Sleep hours - weekdays or workdays",
    "SLQ320": "Usual sleep time on weekends",
    "SLQ330": "Usual wake time on weekends",
    "SLD013": "Sleep hours - weekends"
}

sleep = sleep.rename(columns=sleep_cols)

sleep.head()

File downloaded successfully.


,id,Usual sleep time on weekdays or workdays,Usual wake time on weekdays or workdays,Sleep hours - weekdays or workdays,Usual sleep time on weekends,Usual wake time on weekends,Sleep hours - weekends
0,130378.0,21:30,07:00,9.5,00:00,09:00,9.0
1,130379.0,21:00,06:00,9.0,21:00,06:00,9.0
2,130380.0,00:00,08:00,8.0,00:00,09:00,9.0
3,130384.0,21:30,05:00,7.5,23:00,07:00,8.0
4,130385.0,22:05,06:15,8.0,22:05,06:15,8.0


In [649]:
sleep.describe()

,id,Sleep hours - weekdays or workdays,Sleep hours - weekends
count,8501.000000,8388.000000,8387.000000
mean,136369.136572,7.757332,8.353762
std,3443.283289,1.616056,1.730015
min,130378.000000,2.000000,2.000000
25%,133378.000000,7.000000,7.500000
50%,136402.000000,8.000000,8.500000
75%,139342.000000,8.500000,9.000000
max,142310.000000,14.000000,14.000000


In [650]:
sleep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8501 entries, 0 to 8500
Data columns (total 7 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        8501 non-null   float64
 1   Usual sleep time on weekdays or workdays  8501 non-null   object 
 2   Usual wake time on weekdays or workdays   8501 non-null   object 
 3   Sleep hours - weekdays or workdays        8388 non-null   float64
 4   Usual sleep time on weekends              8501 non-null   object 
 5   Usual wake time on weekends               8501 non-null   object 
 6   Sleep hours - weekends                    8387 non-null   float64
dtypes: float64(3), object(4)
memory usage: 465.0+ KB


In [651]:
# Alcohol

alcohol_cols = {
    "SEQN": "id",
    "ALQ111": "Ever had a drink of any kind of alcohol",
    "ALQ121": "Past 12 months how often drink alcohol beverages",
    "ALQ130": "avg_drinks_day", # Average # alcoholic drinks per day (past 12 months)
    "ALQ142": "# days had 4/5 drinks (past 12 months)",
    "ALQ270": "# times had 4/5 drinks in 2 hours (past 12 months)",
    "ALQ280": "# times had 8+ drinks in 1 day (past 12 months)",
    "ALQ151": "Ever had 4/5 or more drinks every day",
    "ALQ170": "# times had 4/5 drinks on occasion (past month)"
}

url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/ALQ_L.xpt'

file_name = "ALQ_L.xpt"

response = requests.get(url)
if response.status_code == 200:
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

alcohol, meta_alcohol = pyreadstat.read_xport(file_name, encoding='latin1')

alcohol = alcohol.rename(columns=alcohol_cols)

alcohol.head()

File downloaded successfully.


,id,Ever had a drink of any kind of alcohol,Past 12 months how often drink alcohol beverages,avg_drinks_day,# days had 4/5 drinks (past 12 months),# times had 4/5 drinks in 2 hours (past 12 months),# times had 8+ drinks in 1 day (past 12 months),Ever had 4/5 or more drinks every day,# times had 4/5 drinks on occasion (past month)
0,130378.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,130379.0,1.0,2.0,3.0,0.0,NaN,NaN,2.0,NaN
2,130380.0,1.0,10.0,1.0,0.0,NaN,NaN,2.0,NaN
3,130386.0,1.0,4.0,2.0,10.0,0.0,10.0,2.0,0.0
4,130387.0,1.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN


In [652]:
alcohol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6337 entries, 0 to 6336
Data columns (total 9 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   id                                                  6337 non-null   float64
 1   Ever had a drink of any kind of alcohol             5481 non-null   float64
 2   Past 12 months how often drink alcohol beverages    4922 non-null   float64
 3   avg_drinks_day                                      4069 non-null   float64
 4   # days had 4/5 drinks (past 12 months)              4082 non-null   float64
 5   # times had 4/5 drinks in 2 hours (past 12 months)  2366 non-null   float64
 6   # times had 8+ drinks in 1 day (past 12 months)     2362 non-null   float64
 7   Ever had 4/5 or more drinks every day               4901 non-null   float64
 8   # times had 4/5 drinks on occasion (past month)     2358 non-null   float64
dty

In [653]:
alcohol.describe()

,id,Ever had a drink of any kind of alcohol,Past 12 months how often drink alcohol beverages,avg_drinks_day,# days had 4/5 drinks (past 12 months),# times had 4/5 drinks in 2 hours (past 12 months),# times had 8+ drinks in 1 day (past 12 months),Ever had 4/5 or more drinks every day,# times had 4/5 drinks on occasion (past month)
count,6337.000000,5481.000000,4922.000000,4069.000000,4082.000000,2366.000000,2362.000000,4901.000000,2358.000000
mean,136348.738362,1.109104,5.030679,5.842959,4.742283,4.838123,3.545301,1.821261,4.396098
std,3439.132476,0.385114,4.314321,54.996448,7.326042,7.785415,7.133496,0.458352,45.252453
min,130378.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,133337.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000
50%,136397.000000,1.000000,5.000000,2.000000,4.000000,4.000000,0.000000,2.000000,1.000000
75%,139310.000000,1.000000,8.000000,3.000000,9.000000,9.000000,7.000000,2.000000,2.000000
max,142310.000000,9.000000,99.000000,999.000000,99.000000,99.000000,99.000000,9.000000,999.000000


In [654]:
alcohol.avg_drinks_day.value_counts()

avg_drinks_day
1.0      1345
2.0      1312
3.0       599
4.0       251
5.0       216
6.0       141
12.0       55
8.0        42
10.0       31
15.0       20
9.0        19
7.0        17
999.0      10
777.0       4
11.0        4
14.0        2
13.0        1
Name: count, dtype: int64

In [655]:
alcohol = alcohol.loc[~alcohol['avg_drinks_day'].isin([777, 999])]
alcohol.head()

,id,Ever had a drink of any kind of alcohol,Past 12 months how often drink alcohol beverages,avg_drinks_day,# days had 4/5 drinks (past 12 months),# times had 4/5 drinks in 2 hours (past 12 months),# times had 8+ drinks in 1 day (past 12 months),Ever had 4/5 or more drinks every day,# times had 4/5 drinks on occasion (past month)
0,130378.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,130379.0,1.0,2.0,3.0,0.0,NaN,NaN,2.0,NaN
2,130380.0,1.0,10.0,1.0,0.0,NaN,NaN,2.0,NaN
3,130386.0,1.0,4.0,2.0,10.0,0.0,10.0,2.0,0.0
4,130387.0,1.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN


In [656]:
alcohol = alcohol.dropna()

In [657]:
alcohol.describe()

,id,Ever had a drink of any kind of alcohol,Past 12 months how often drink alcohol beverages,avg_drinks_day,# days had 4/5 drinks (past 12 months),# times had 4/5 drinks in 2 hours (past 12 months),# times had 8+ drinks in 1 day (past 12 months),Ever had 4/5 or more drinks every day,# times had 4/5 drinks on occasion (past month)
count,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000
mean,136339.321627,1.003426,5.188009,3.370450,7.990150,4.726338,3.501071,1.814561,3.658244
std,3471.462772,0.165557,3.780416,2.421568,7.014759,7.280349,6.886326,0.472330,37.251129
min,130386.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,133235.500000,1.000000,3.000000,2.000000,6.000000,0.000000,0.000000,2.000000,0.000000
50%,136354.000000,1.000000,5.000000,3.000000,8.000000,4.000000,0.000000,2.000000,1.000000
75%,139375.000000,1.000000,7.000000,4.000000,10.000000,9.000000,7.000000,2.000000,2.000000
max,142310.000000,9.000000,99.000000,15.000000,99.000000,99.000000,99.000000,9.000000,999.000000


In [658]:
# Blood pressure

url_bp = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.xpt'

file_name_bp = "BPXO_L.xpt"

response = requests.get(url_bp)
if response.status_code == 200:
    with open(file_name_bp, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

blood_pressure, meta_bp = pyreadstat.read_xport(file_name_bp)

bp_cols = {
    'SEQN' : 'id',
    'BPAOARM' : 'Arm selected - oscillometric',
    'BPAOCSZ' : 'Coded cuff size - oscillometric',
    'BPXOSY1' : 'Systolic - 1st oscillometric reading',
    'BPXODI1' : 'Diastolic - 1st oscillometric reading',
    'BPXOSY2' : 'Systolic - 2nd oscillometric reading',
    'BPXODI2' : 'Diastolic - 2nd oscillometric reading',
    'BPXOSY3' : 'Systolic - 3rd oscillometric reading',
    'BPXODI3' : 'Diastolic - 3rd oscillometric reading',
    'BPXOPLS1' : 'Pulse - 1st oscillometric reading',
    'BPXOPLS2' : 'Pulse - 2nd oscillometric reading',
    'BPXOPLS3' : 'Pulse - 3rd oscillometric reading',
}

blood_pressure = blood_pressure.rename(columns=bp_cols)

blood_pressure.head()

File downloaded successfully.


,id,Arm selected - oscillometric,Coded cuff size - oscillometric,Systolic - 1st oscillometric reading,Diastolic - 1st oscillometric reading,Systolic - 2nd oscillometric reading,Diastolic - 2nd oscillometric reading,Systolic - 3rd oscillometric reading,Diastolic - 3rd oscillometric reading,Pulse - 1st oscillometric reading,Pulse - 2nd oscillometric reading,Pulse - 3rd oscillometric reading
0,130378.0,R,4.0,135.0,98.0,131.0,96.0,132.0,94.0,82.0,79.0,82.0
1,130379.0,R,4.0,121.0,84.0,117.0,76.0,113.0,76.0,72.0,71.0,73.0
2,130380.0,R,4.0,111.0,79.0,112.0,80.0,104.0,76.0,84.0,83.0,77.0
3,130386.0,R,4.0,110.0,72.0,120.0,74.0,115.0,75.0,59.0,64.0,64.0
4,130387.0,R,4.0,143.0,76.0,136.0,74.0,145.0,78.0,80.0,80.0,77.0


In [659]:
# Creating averages of the three readings for blood pressure

blood_pressure['avg_pulse'] = blood_pressure[['Pulse - 1st oscillometric reading', 'Pulse - 2nd oscillometric reading', 'Pulse - 3rd oscillometric reading']].mean(axis=1)
blood_pressure['avg_systolic'] = blood_pressure[['Systolic - 1st oscillometric reading','Systolic - 2nd oscillometric reading','Systolic - 3rd oscillometric reading']].mean(axis=1)
blood_pressure['avg_diastolic'] = blood_pressure[['Diastolic - 1st oscillometric reading','Diastolic - 2nd oscillometric reading','Diastolic - 3rd oscillometric reading']].mean(axis=1)
blood_pressure = blood_pressure[['id','avg_systolic','avg_diastolic','avg_pulse']]
blood_pressure.describe()

,id,avg_systolic,avg_diastolic,avg_pulse
count,7801.000000,7518.000000,7518.000000,7518.000000
mean,136349.487117,119.094418,72.207280,73.041789
std,3449.490842,18.151729,11.471177,12.564442
min,130378.000000,70.000000,34.000000,34.000000
25%,133335.000000,106.333333,64.000000,64.333333
50%,136382.000000,116.333333,71.666667,72.166667
75%,139325.000000,129.000000,79.333333,81.000000
max,142310.000000,232.333333,139.000000,151.000000


In [660]:
# Total nutrient intake, Day 1

url_tn ='https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR1TOT_L.xpt'

file_name_tn = "DR1IFF_L.xpt"

response = requests.get(url_tn)
if response.status_code == 200:
    with open(file_name_tn, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

total_nutrients_1, meta_tn = pyreadstat.read_xport(file_name_tn)

tn_cols = {
    "SEQN": "id",
    "WTDRD1": "Dietary day one sample weight",
    "WTDR2D": "Dietary two-day sample weight",
    "DR1DRSTZ": "Dietary recall status",
    "DR1EXMER": "Interviewer ID code",
    "DRABF": "Breast-fed infant (either day)",
    "DRDINT": "Number of days of intake",
    "DR1DBIH": "# of days b/w intake and HH interview",
    "DR1DAY": "Intake day of the week",
    "DR1LANG": "Language respondent used mostly",
    "DR1MRESP": "Main respondent for this interview",
    "DR1HELP": "Helped in responding for this interview",
    "DBQ095Z": "Type of table salt used",
    "DBD100": "How often add salt to food at table",
    "DRQSPREP": "Salt used in preparation?",
    "DR1STY": "Salt used at table yesterday?",
    "DR1SKY": "Type of salt used yesterday",
    "DRQSDIET": "On special diet?",
    "DRQSDT1": "Weight loss/Low calorie diet",
    "DRQSDT2": "Low fat/Low cholesterol diet",
    "DRQSDT3": "Low salt/Low sodium diet",
    "DRQSDT4": "Sugar free/Low sugar diet",
    "DRQSDT5": "Low fiber diet",
    "DRQSDT6": "High fiber diet",
    "DRQSDT7": "Diabetic diet",
    "DRQSDT8": "Weight gain/Muscle building diet",
    "DRQSDT9": "Low carbohydrate diet",
    "DRQSDT10": "High protein diet",
    "DRQSDT11": "Gluten-free/Celiac diet",
    "DRQSDT12": "Renal/Kidney diet",
    "DRQSDT91": "Other special diet",
    "DR1TNUMF": "Number of foods/beverages reported",
    "DR1TKCAL": "Energy (kcal)",
    "DR1TPROT": "Protein (gm)",
    "DR1TCARB": "Carbohydrate (gm)",
    "DR1TSUGR": "Total sugars (gm)",
    "DR1TFIBE": "Dietary fiber (gm)",
    "DR1TTFAT": "Total fat (gm)",
    "DR1TSFAT": "Total saturated fatty acids (gm)",
    "DR1TMFAT": "Total monounsaturated fatty acids (gm)",
    "DR1TPFAT": "Total polyunsaturated fatty acids (gm)",
    "DR1TCHOL": "Cholesterol (mg)",
    "DR1TATOC": "Vitamin E as alpha-tocopherol (mg)",
    "DR1TATOA": "Added alpha-tocopherol (Vitamin E) (mg)",
    "DR1TRET": "Retinol (mcg)",
    "DR1TVARA": "Vitamin A, RAE (mcg)",
    "DR1TACAR": "Alpha-carotene (mcg)",
    "DR1TBCAR": "Beta-carotene (mcg)",
    "DR1TCRYP": "Beta-cryptoxanthin (mcg)",
    "DR1TLYCO": "Lycopene (mcg)",
    "DR1TLZ": "Lutein + zeaxanthin (mcg)",
    "DR1TVB1": "Thiamin (Vitamin B1) (mg)",
    "DR1TVB2": "Riboflavin (Vitamin B2) (mg)",
    "DR1TNIAC": "Niacin (mg)",
    "DR1TVB6": "Vitamin B6 (mg)",
    "DR1TFOLA": "Total folate (mcg)",
    "DR1TFA": "Folic acid (mcg)",
    "DR1TFF": "Food folate (mcg)",
    "DR1TFDFE": "Folate, DFE (mcg)",
    "DR1TCHL": "Total choline (mg)",
    "DR1TVB12": "Vitamin B12 (mcg)",
    "DR1TB12A": "Added vitamin B12 (mcg)",
    "DR1TVC": "Vitamin C (mg)",
    "DR1TVD": "Vitamin D (D2 + D3) (mcg)",
    "DR1TVK": "Vitamin K (mcg)",
    "DR1TCALC": "Calcium (mg)",
    "DR1TPHOS": "Phosphorus (mg)",
    "DR1TMAGN": "Magnesium (mg)",
    "DR1TIRON": "Iron (mg)",
    "DR1TZINC": "Zinc (mg)",
    "DR1TCOPP": "Copper (mg)",
    "DR1TSODI": "Sodium (mg)",
    "DR1TPOTA": "Potassium (mg)",
    "DR1TSELE": "Selenium (mcg)",
    "DR1TCAFF": "Caffeine (mg)",
    "DR1TTHEO": "Theobromine (mg)",
    "DR1TALCO": "Alcohol (gm)",
    "DR1TMOIS": "Moisture (gm)",
    "DR1TS040": "SFA 4:0 (Butanoic) (gm)",
    "DR1TS060": "SFA 6:0 (Hexanoic) (gm)",
    "DR1TS080": "SFA 8:0 (Octanoic) (gm)",
    "DR1TS100": "SFA 10:0 (Decanoic) (gm)",
    "DR1TS120": "SFA 12:0 (Dodecanoic) (gm)",
    "DR1TS140": "SFA 14:0 (Tetradecanoic) (gm)",
    "DR1TS160": "SFA 16:0 (Hexadecanoic) (gm)",
    "DR1TS180": "SFA 18:0 (Octadecanoic) (gm)",
    "DR1TM161": "MFA 16:1 (Hexadecenoic) (gm)",
    "DR1TM181": "MFA 18:1 (Octadecenoic) (gm)",
    "DR1TM201": "MFA 20:1 (Eicosenoic) (gm)",
    "DR1TM221": "MFA 22:1 (Docosenoic) (gm)",
    "DR1TP182": "PFA 18:2 (Octadecadienoic) (gm)",
    "DR1TP183": "PFA 18:3 (Octadecatrienoic) (gm)",
    "DR1TP184": "PFA 18:4 (Octadecatetraenoic) (gm)",
    "DR1TP204": "PFA 20:4 (Eicosatetraenoic) (gm)",
    "DR1TP205": "PFA 20:5 (Eicosapentaenoic) (gm)",
    "DR1TP225": "PFA 22:5 (Docosapentaenoic) (gm)",
    "DR1TP226": "PFA 22:6 (Docosahexaenoic) (gm)",
    "DR1_300": "Compare food consumed yesterday to usual",
    "DR1_320Z": "Total plain water drank yesterday (gm)",
    "DR1_330Z": "Total tap water drank yesterday (gm)",
    "DR1BWATZ": "Total bottled water drank yesterday (gm)",
    "DR1TWSZ": "Tap water source",
    "DRD340": "Shellfish eaten during past 30 days",
    "DRD350A": "Clams eaten during past 30 days",
    "DRD350AQ": "# of times clams eaten in past 30 days",
    "DRD350B": "Crabs eaten during past 30 days",
    "DRD350BQ": "# of times crabs eaten in past 30 days",
    "DRD350C": "Crayfish eaten during past 30 days",
    "DRD350CQ": "# of times crayfish eaten past 30 days",
    "DRD350D": "Lobsters eaten during past 30 days",
    "DRD350DQ": "# of times lobsters eaten past 30 days",
    "DRD350E": "Mussels eaten during past 30 days",
    "DRD350EQ": "# of times mussels eaten in past 30 days",
    "DRD350F": "Oysters eaten during past 30 days",
    "DRD350FQ": "# of times oysters eaten in past 30 days",
    "DRD350G": "Scallops eaten during past 30 days",
    "DRD350GQ": "# of times scallops eaten past 30 days",
    "DRD350H": "Shrimp eaten during past 30 days",
    "DRD350HQ": "# of times shrimp eaten in past 30 days",
    "DRD350I": "Other shellfish eaten past 30 days",
    "DRD350IQ": "# of times other shellfish eaten",
    "DRD350J": "Other unknown shellfish eaten past 30 days",
    "DRD350JQ": "# of times other unknown shellfish eaten",
    "DRD350K": "Refused on shellfish eaten past 30 days",
    "DRD360": "Fish eaten during past 30 days",
    "DRD370A": "Breaded fish products eaten past 30 days",
    "DRD370AQ": "# of times breaded fish products eaten",
    "DRD370B": "Tuna eaten during past 30 days",
    "DRD370BQ": "# of times tuna eaten in past 30 days",
    "DRD370C": "Bass eaten during past 30 days",
    "DRD370CQ": "# of times bass eaten in past 30 days",
    "DRD370D": "Catfish eaten during past 30 days",
    "DRD370DQ": "# of times catfish eaten in past 30 days",
    "DRD370E": "Cod eaten during past 30 days",
    "DRD370EQ": "# of times cod eaten in past 30 days",
    "DRD370F": "Flatfish eaten during past 30 days",
    "DRD370FQ": "# of times flatfish eaten past 30 days",
    "DRD370G": "Haddock eaten during past 30 days",
    "DRD370GQ": "# of times haddock eaten in past 30 days",
    "DRD370H": "Mackerel eaten during past 30 days",
    "DRD370HQ": "# of times mackerel eaten past 30 days",
    "DRD370I": "Perch eaten during past 30 days",
    "DRD370IQ": "# of times perch eaten in past 30 days",
    "DRD370J": "Pike eaten during past 30 days",
    "DRD370JQ": "# of times pike eaten in past 30 days",
    "DRD370K": "Pollock eaten during past 30 days",
    "DRD370KQ": "# of times pollock eaten in past 30 days",
    "DRD370L": "Porgy eaten during past 30 days",
    "DRD370LQ": "# of times porgy eaten in past 30 days",
    "DRD370M": "Salmon eaten during past 30 days",
    "DRD370MQ": "# of times salmon eaten in past 30 days",
    "DRD370N": "Sardines eaten during past 30 days",
    "DRD370NQ": "# of times sardines eaten past 30 days",
    "DRD370O": "Sea bass eaten during past 30 days",
    "DRD370OQ": "# of times sea bass eaten past 30 days",
    "DRD370P": "Shark eaten during past 30 days",
    "DRD370PQ": "# of times shark eaten in past 30 days",
    "DRD370Q": "Swordfish eaten during past 30 days",
    "DRD370QQ": "# of times swordfish eaten past 30 days",
    "DRD370R": "Trout eaten during past 30 days",
    "DRD370RQ": "# of times trout eaten in past 30 days",
    "DRD370S": "Walleye eaten during past 30 days",
    "DRD370SQ": "# of times walleye eaten in past 30 days",
    "DRD370T": "Other fish eaten during past 30 days",
    "DRD370TQ": "# of times other fish eaten past 30 days",
    "DRD370U": "Other unknown fish eaten in past 30 days",
    "DRD370UQ": "# of times other unknown fish eaten",
    "DRD370V": "Refused on fish eaten past 30 days"
}

total_nutrients_1 = total_nutrients_1.rename(columns=tn_cols)

total_nutrients_1.head()

File downloaded successfully.


,id,Dietary day one sample weight,Dietary two-day sample weight,Dietary recall status,Interviewer ID code,Breast-fed infant (either day),Number of days of intake,# of days b/w intake and HH interview,Intake day of the week,Language respondent used mostly,...,# of times swordfish eaten past 30 days,Trout eaten during past 30 days,# of times trout eaten in past 30 days,Walleye eaten during past 30 days,# of times walleye eaten in past 30 days,Other fish eaten during past 30 days,# of times other fish eaten past 30 days,Other unknown fish eaten in past 30 days,# of times other unknown fish eaten,Refused on fish eaten past 30 days
0,130378.0,61366.555827,70554.222162,1.0,73.0,2.0,2.0,40.0,4.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
1,130379.0,34638.056480,36505.468348,1.0,73.0,2.0,2.0,19.0,4.0,1.0,...,1.0,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
2,130380.0,84728.261560,103979.190677,1.0,73.0,2.0,2.0,16.0,4.0,1.0,...,NaN,2.0,NaN,2.0,NaN,1.0,4.0,2.0,NaN,2.0
3,130381.0,61737.133446,75009.220819,1.0,91.0,2.0,2.0,23.0,5.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
4,130382.0,75846.746917,172361.851828,1.0,73.0,2.0,2.0,27.0,6.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0


In [661]:
# Income

url_inc ='https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/INQ_L.xpt'

file_name_inc = "INQ_L.xpt"

response = requests.get(url_inc)
if response.status_code == 200:
    with open(file_name_inc, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

income, meta_inc = pyreadstat.read_xport(file_name_inc)

inc_cols = {
    "SEQN": "id",
    "INDFMMPI": "Family monthly poverty level index",
    "INDFMMPC": "Family monthly poverty level category",
    "INQ300": "Family has savings more than $20,000",
    "IND310": "Total savings/cash assets for the family"
}

income = income.rename(columns=inc_cols)

income.head()

File downloaded successfully.


,id,Family monthly poverty level index,Family monthly poverty level category,"Family has savings more than $20,000",Total savings/cash assets for the family
0,130378.0,5.00,3.0,1.0,NaN
1,130379.0,5.00,3.0,1.0,NaN
2,130380.0,1.40,2.0,2.0,1.0
3,130381.0,0.33,1.0,2.0,1.0
4,130382.0,4.32,3.0,1.0,NaN


In [662]:
income = income.drop(['Total savings/cash assets for the family'], axis=1)

In [663]:
income = income[income['Family monthly poverty level index'] <= 3.0]
income.shape

(5697, 4)

In [664]:
# Body measurements

url_body = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.xpt'

file_name_body = 'BMX_L.xpt'

response = requests.get(url_body)
if response.status_code == 200:
    with open(file_name_body, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

body, meta_body = pyreadstat.read_xport(file_name_body)

body_cols = {
    "SEQN": "id",
    "BMDSTATS": "Body Measures Component Status Code",
    "BMXWT": "Weight (kg)",
    "BMIWT": "Weight Comment",
    "BMXRECUM": "Recumbent Length (cm)",
    "BMIRECUM": "Recumbent Length Comment",
    "BMXHEAD": "Head Circumference (cm)",
    "BMIHEAD": "Head Circumference Comment",
    "BMXHT": "Standing Height (cm)",
    "BMIHT": "Standing Height Comment",
    "BMXBMI": "bmi",
    "BMDBMIC": "BMI Category - Children/Youth",
    "BMXLEG": "Upper Leg Length (cm)",
    "BMILEG": "Upper Leg Length Comment",
    "BMXARML": "Upper Arm Length (cm)",
    "BMIARML": "Upper Arm Length Comment",
    "BMXARMC": "Arm Circumference (cm)",
    "BMIARMC": "Arm Circumference Comment",
    "BMXWAIST": "Waist Circumference (cm)",
    "BMIWAIST": "Waist Circumference Comment",
    "BMXHIP": "Hip Circumference (cm)",
    "BMIHIP": "Hip Circumference Comment"
}

body = body.rename(columns=body_cols)

body = body[['id','bmi']]

File downloaded successfully.


In [665]:
# # Insulin

# url_insulin = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/INS_L.xpt'

# file_name_insulin = 'INS_L.xpt'

# response = requests.get(url_insulin)
# if response.status_code == 200:
#     with open(file_name_insulin, 'wb') as f:
#         f.write(response.content)
#     print("File downloaded successfully.")
# else:
#     print("Failed to download file.")

# insulin, meta_insulin = pyreadstat.read_xport(file_name_insulin)

# insulin_cols = {
#     'SEQN':'id',
#     'LBXIN': 'Insulin (uU/mL)',
#     'LBDINSI':'Insulin (pmol/L)'
# }

# insulin = insulin.rename(columns=insulin_cols)

# insulin = insulin[['id','Insulin (uU/mL)','Insulin (pmol/L)']]

# insulin.head()



In [666]:
# # Mercury

# """SEQN - Respondent sequence number
# WTPH2YR - Phlebotomy 2 Year Weight
# LBXIHG - Mercury, inorganic (ug/L)
# LBDIHGSI - Mercury, inorganic (nmol/L)
# LBDIHGLC - Mercury, inorganic comment code
# LBXBGE - Mercury, ethyl (ug/L)
# LBDBGESI - Mercury, ethyl (nmol/L)
# LBDBGELC - Mercury, ethyl comment code
# LBXBGM - Mercury, methyl (ug/L)
# LBDBGMSI - Mercury, methyl (nmol/L)
# LBDBGMLC - Mercury, methyl comment code"""

# url_merc = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/IHGEM_L.xpt'

# file_name_merc = 'IHGEM_L.xpt'

# response = requests.get(url_merc)
# if response.status_code == 200:
#     with open(file_name_merc, 'wb') as f:
#         f.write(response.content)
#     print("File downloaded successfully.")
# else:
#     print("Failed to download file.")

# mercury, meta_merc = pyreadstat.read_xport(file_name_merc)

# merc_cols = {
#     'SEQN':'id',
#     'LBXBGM':'Mercury, methyl (ug/L)'
# }

# mercury = mercury.rename(columns=merc_cols)

# mercury.head()

In [667]:
# # Depression

# depression_cols = {
#     "SEQN": "id",
#     "DPQ010": "Have little interest in doing things",
#     "DPQ020": "Feeling down, depressed, or hopeless",
#     "DPQ030": "Trouble sleeping or sleeping too much",
#     "DPQ040": "Feeling tired or having little energy",
#     "DPQ050": "Poor appetite or overeating",
#     "DPQ060": "Feeling bad about yourself",
#     "DPQ070": "Trouble concentrating on things",
#     "DPQ080": "Moving or speaking slowly or too fast",
#     "DPQ090": "Thought you would be better off dead",
#     "DPQ100": "Difficulty these problems have caused"
# }

# url_depression = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DPQ_L.xpt'

# file_name_depression = 'DPQ_L.xpt'

# response = requests.get(url_depression)
# if response.status_code == 200:
#     with open(file_name_depression, 'wb') as f:
#         f.write(response.content)
#     print("File downloaded successfully.")
# else:
#     print("Failed to download file.")

# depression, meta_depression = pyreadstat.read_xport(file_name_depression)

# depression = depression.rename(columns=depression_cols)

# depression['depression_score'] = depression[[
#     "Have little interest in doing things",
#     "Feeling down, depressed, or hopeless",
#     "Trouble sleeping or sleeping too much",
#     "Feeling tired or having little energy",
#     "Poor appetite or overeating",
#     "Feeling bad about yourself",
#     "Trouble concentrating on things",
#     "Moving or speaking slowly or too fast",
#     "Thought you would be better off dead",
#     "Difficulty these problems have caused"
# ]].sum(axis=1)

# depression.head()

In [668]:
# Physical activity

phys_cols = {
    "SEQN": "id",
    "PAD790Q": "Frequency of moderate LTPA",
    "PAD790U": "Moderate LTPA unit (day/week/month/year)",
    "PAD800": "Minutes moderate LTPA",
    "PAD810Q": "Frequency of vigorous LTPA",
    "PAD810U": "Vigorous LTPA unit (day/week/month/year)",
    "PAD820": "Minutes vigorous LTPA",
    "PAD680": "Minutes sedentary activity"
}

url_phys = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/PAQ_L.xpt'
file_name_phys = 'PAQ_L.xpt'

response = requests.get(url_phys)
if response.status_code == 200:
    with open(file_name_phys, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

physical_activity, meta_phys = pyreadstat.read_xport(file_name_phys)

physical_activity = physical_activity.rename(columns=phys_cols)

physical_activity.head()

File downloaded successfully.


,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0
2,130380.0,1.0,W,20.0,0.0,,NaN,240.0
3,130384.0,0.0,,NaN,0.0,,NaN,60.0
4,130385.0,1.0,D,90.0,1.0,W,60.0,180.0


In [669]:
physical_activity[physical_activity['Frequency of moderate LTPA'] == 9999]

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity
119,130552.0,9999.0,,NaN,9999.0,,NaN,360.0
753,131490.0,9999.0,,NaN,0.0,,NaN,120.0
765,131510.0,9999.0,,NaN,9999.0,,NaN,9999.0
1055,131932.0,9999.0,,NaN,9999.0,,NaN,180.0
1211,132171.0,9999.0,,NaN,0.0,,NaN,120.0
1478,132563.0,9999.0,,NaN,9999.0,,NaN,60.0
1550,132677.0,9999.0,,NaN,9999.0,,NaN,240.0
1812,133058.0,9999.0,,NaN,0.0,,NaN,240.0
2385,133928.0,9999.0,,NaN,0.0,,NaN,9999.0
2415,133974.0,9999.0,,NaN,0.0,,NaN,600.0


In [670]:
physical_activity = physical_activity.loc[~physical_activity['Frequency of moderate LTPA'].isin([7777, 9999])]
physical_activity.head()

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0
2,130380.0,1.0,W,20.0,0.0,,NaN,240.0
3,130384.0,0.0,,NaN,0.0,,NaN,60.0
4,130385.0,1.0,D,90.0,1.0,W,60.0,180.0


In [671]:
physical_activity['Moderate LTPA unit (day/week/month/year)'].value_counts()

Moderate LTPA unit (day/week/month/year)
W    4851
     1714
D     985
M     472
Y      82
Name: count, dtype: int64

In [672]:
physical_activity[physical_activity['Moderate LTPA unit (day/week/month/year)'] == 'W']

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0
2,130380.0,1.0,W,20.0,0.0,,NaN,240.0
5,130386.0,1.0,W,30.0,1.0,M,30.0,180.0
7,130388.0,5.0,W,180.0,0.0,,NaN,360.0
...,...,...,...,...,...,...,...,...
8138,142293.0,2.0,W,15.0,0.0,,NaN,300.0
8141,142297.0,3.0,W,15.0,5.0,M,45.0,300.0
8148,142305.0,2.0,W,40.0,0.0,,NaN,480.0
8149,142307.0,3.0,W,15.0,0.0,,NaN,480.0


In [673]:
# Creating columns in physical_activity for minutes of exercise per week

freq_mod = physical_activity['Frequency of moderate LTPA']
unit_mod = physical_activity['Moderate LTPA unit (day/week/month/year)']
min_mod = physical_activity['Minutes moderate LTPA']

min_mod_week = np.where(unit_mod == 'D', freq_mod * min_mod * 7,
               np.where(unit_mod == 'W', freq_mod * min_mod,
               np.where(unit_mod == 'M', freq_mod / 4 * min_mod,
               np.where(unit_mod == 'Y', freq_mod / 52 * min_mod,
               np.nan))))

physical_activity['min_mod_week'] = min_mod_week

physical_activity.head(20)

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity,min_mod_week
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0,135.0
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0,180.0
2,130380.0,1.0,W,20.0,0.0,,NaN,240.0,20.0
3,130384.0,0.0,,NaN,0.0,,NaN,60.0,NaN
4,130385.0,1.0,D,90.0,1.0,W,60.0,180.0,630.0
5,130386.0,1.0,W,30.0,1.0,M,30.0,180.0,30.0
6,130387.0,0.0,,NaN,0.0,,NaN,1200.0,NaN
7,130388.0,5.0,W,180.0,0.0,,NaN,360.0,900.0
8,130389.0,3.0,W,45.0,2.0,M,30.0,720.0,135.0
9,130390.0,1.0,W,60.0,0.0,,NaN,300.0,60.0


In [674]:
freq_vig = physical_activity['Frequency of vigorous LTPA']
unit_vig = physical_activity['Vigorous LTPA unit (day/week/month/year)']
min_vig = physical_activity['Minutes vigorous LTPA']

min_vig_week = np.where(unit_vig == 'D', freq_vig * min_vig * 7,
               np.where(unit_vig == 'W', freq_vig * min_vig,
               np.where(unit_vig == 'M', freq_vig / 4 * min_vig,
               np.where(unit_vig == 'Y', freq_vig / 52 * min_vig,
               np.nan))))

physical_activity['min_vig_week'] = min_vig_week

physical_activity.head(20)

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity,min_mod_week,min_vig_week
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0,135.0,135.000000
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0,180.0,135.000000
2,130380.0,1.0,W,20.0,0.0,,NaN,240.0,20.0,NaN
3,130384.0,0.0,,NaN,0.0,,NaN,60.0,NaN,NaN
4,130385.0,1.0,D,90.0,1.0,W,60.0,180.0,630.0,60.000000
5,130386.0,1.0,W,30.0,1.0,M,30.0,180.0,30.0,7.500000
6,130387.0,0.0,,NaN,0.0,,NaN,1200.0,NaN,NaN
7,130388.0,5.0,W,180.0,0.0,,NaN,360.0,900.0,NaN
8,130389.0,3.0,W,45.0,2.0,M,30.0,720.0,135.0,15.000000
9,130390.0,1.0,W,60.0,0.0,,NaN,300.0,60.0,NaN


In [675]:
physical_activity.dropna()

,id,Frequency of moderate LTPA,Moderate LTPA unit (day/week/month/year),Minutes moderate LTPA,Frequency of vigorous LTPA,Vigorous LTPA unit (day/week/month/year),Minutes vigorous LTPA,Minutes sedentary activity,min_mod_week,min_vig_week
0,130378.0,3.0,W,45.0,3.0,W,45.0,360.0,135.0,135.00
1,130379.0,4.0,W,45.0,3.0,W,45.0,480.0,180.0,135.00
4,130385.0,1.0,D,90.0,1.0,W,60.0,180.0,630.0,60.00
5,130386.0,1.0,W,30.0,1.0,M,30.0,180.0,30.0,7.50
8,130389.0,3.0,W,45.0,2.0,M,30.0,720.0,135.0,15.00
...,...,...,...,...,...,...,...,...,...,...
8136,142289.0,3.0,W,60.0,3.0,W,60.0,150.0,180.0,180.00
8137,142290.0,1.0,D,60.0,2.0,W,60.0,240.0,420.0,120.00
8141,142297.0,3.0,W,15.0,5.0,M,45.0,300.0,45.0,56.25
8143,142299.0,1.0,D,60.0,1.0,D,30.0,120.0,420.0,210.00


In [676]:
# Nutrients day 2 

url = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DR2TOT_L.xpt'

file_name = 'DR2TOT_L.xpt'

response = requests.get(url)
if response.status_code == 200:
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")

day_2, meta_day_2 = pyreadstat.read_xport(file_name)

day_2_cols = {
    "SEQN": "id",
    "WTDRD1": "Dietary day one sample weight",
    "WTDR2D": "Dietary two-day sample weight",
    "DR2DRSTZ": "Dietary recall status",
    "DR2EXMER": "Interviewer ID code",
    "DRABF": "Breast-fed infant (either day)",
    "DRDINT": "Number of days of intake",
    "DR2DBIH": "# of days between intake and HH interview",
    "DR2DAY": "Intake day of the week",
    "DR2LANG": "Language respondent used mostly",
    "DR2MRESP": "Main respondent for this interview",
    "DR2HELP": "Helped in responding for this interview",
    "DR2TNUMF": "Number of foods/beverages reported",
    "DR2STY": "Salt used at table yesterday?",
    "DR2SKY": "Type of salt used yesterday",
    "DR2TKCAL": "Energy (kcal)",
    "DR2TPROT": "Protein (gm)",
    "DR2TCARB": "Carbohydrate (gm)",
    "DR2TSUGR": "Total sugars (gm)",
    "DR2TFIBE": "Dietary fiber (gm)",
    "DR2TTFAT": "Total fat (gm)",
    "DR2TSFAT": "Total saturated fatty acids (gm)",
    "DR2TMFAT": "Total monounsaturated fatty acids (gm)",
    "DR2TPFAT": "Total polyunsaturated fatty acids (gm)",
    "DR2TCHOL": "Cholesterol (mg)",
    "DR2TATOC": "Vitamin E as alpha-tocopherol (mg)",
    "DR2TATOA": "Added alpha-tocopherol (Vitamin E) (mg)",
    "DR2TRET": "Retinol (mcg)",
    "DR2TVARA": "Vitamin A, RAE (mcg)",
    "DR2TACAR": "Alpha-carotene (mcg)",
    "DR2TBCAR": "Beta-carotene (mcg)",
    "DR2TCRYP": "Beta-cryptoxanthin (mcg)",
    "DR2TLYCO": "Lycopene (mcg)",
    "DR2TLZ": "Lutein + zeaxanthin (mcg)",
    "DR2TVB1": "Thiamin (Vitamin B1) (mg)",
    "DR2TVB2": "Riboflavin (Vitamin B2) (mg)",
    "DR2TNIAC": "Niacin (mg)",
    "DR2TVB6": "Vitamin B6 (mg)",
    "DR2TFOLA": "Total folate (mcg)",
    "DR2TFA": "Folic acid (mcg)",
    "DR2TFF": "Food folate (mcg)",
    "DR2TFDFE": "Folate, DFE (mcg)",
    "DR2TCHL": "Total choline (mg)",
    "DR2TVB12": "Vitamin B12 (mcg)",
    "DR2TB12A": "Added vitamin B12 (mcg)",
    "DR2TVC": "Vitamin C (mg)",
    "DR2TVD": "Vitamin D (D2 + D3) (mcg)",
    "DR2TVK": "Vitamin K (mcg)",
    "DR2TCALC": "Calcium (mg)",
    "DR2TPHOS": "Phosphorus (mg)",
    "DR2TMAGN": "Magnesium (mg)",
    "DR2TIRON": "Iron (mg)",
    "DR2TZINC": "Zinc (mg)",
    "DR2TCOPP": "Copper (mg)",
    "DR2TSODI": "Sodium (mg)",
    "DR2TPOTA": "Potassium (mg)",
    "DR2TSELE": "Selenium (mcg)",
    "DR2TCAFF": "Caffeine (mg)",
    "DR2TTHEO": "Theobromine (mg)",
    "DR2TALCO": "Alcohol (gm)",
    "DR2TMOIS": "Moisture (gm)",
    "DR2TS040": "SFA 4:0 (Butanoic) (gm)",
    "DR2TS060": "SFA 6:0 (Hexanoic) (gm)",
    "DR2TS080": "SFA 8:0 (Octanoic) (gm)",
    "DR2TS100": "SFA 10:0 (Decanoic) (gm)",
    "DR2TS120": "SFA 12:0 (Dodecanoic) (gm)",
    "DR2TS140": "SFA 14:0 (Tetradecanoic) (gm)",
    "DR2TS160": "SFA 16:0 (Hexadecanoic) (gm)",
    "DR2TS180": "SFA 18:0 (Octadecanoic) (gm)",
    "DR2TM161": "MFA 16:1 (Hexadecenoic) (gm)",
    "DR2TM181": "MFA 18:1 (Octadecenoic) (gm)",
    "DR2TM201": "MFA 20:1 (Eicosenoic) (gm)",
    "DR2TM221": "MFA 22:1 (Docosenoic) (gm)",
    "DR2TP182": "PFA 18:2 (Octadecadienoic) (gm)",
    "DR2TP183": "PFA 18:3 (Octadecatrienoic) (gm)",
    "DR2TP184": "PFA 18:4 (Octadecatetraenoic) (gm)",
    "DR2TP204": "PFA 20:4 (Eicosatetraenoic) (gm)",
    "DR2TP205": "PFA 20:5 (Eicosapentaenoic) (gm)",
    "DR2TP225": "PFA 22:5 (Docosapentaenoic) (gm)",
    "DR2TP226": "PFA 22:6 (Docosahexaenoic) (gm)",
    "DR2_300": "Compare food consumed yesterday to usual",
    "DR2_320Z": "Total plain water drank yesterday (gm)",
    "DR2_330Z": "Total tap water drank yesterday (gm)",
    "DR2BWATZ": "Total bottled water drank yesterday (gm)",
    "DR2TWSZ": "Tap water source"
}

day_2 = day_2.rename(columns=day_2_cols)

day_2.head()

File downloaded successfully.


,id,Dietary day one sample weight,Dietary two-day sample weight,Dietary recall status,Interviewer ID code,Breast-fed infant (either day),Number of days of intake,# of days between intake and HH interview,Intake day of the week,Language respondent used mostly,...,PFA 18:4 (Octadecatetraenoic) (gm),PFA 20:4 (Eicosatetraenoic) (gm),PFA 20:5 (Eicosapentaenoic) (gm),PFA 22:5 (Docosapentaenoic) (gm),PFA 22:6 (Docosahexaenoic) (gm),Compare food consumed yesterday to usual,Total plain water drank yesterday (gm),Total tap water drank yesterday (gm),Total bottled water drank yesterday (gm),Tap water source
0,130378.0,61366.555827,70554.222162,1.0,49.0,2.0,2.0,45.0,2.0,1.0,...,0.235,0.194,1.500,0.665,1.973,2.0,960.0,960.0,0.0,1.0
1,130379.0,34638.056480,36505.468348,1.0,7.0,2.0,2.0,27.0,5.0,1.0,...,0.000,0.114,0.003,0.007,0.004,2.0,240.0,240.0,0.0,1.0
2,130380.0,84728.261560,103979.190677,1.0,65.0,2.0,2.0,20.0,1.0,1.0,...,0.000,0.092,0.000,0.010,0.009,2.0,960.0,960.0,0.0,1.0
3,130381.0,61737.133446,75009.220819,1.0,91.0,2.0,2.0,40.0,1.0,1.0,...,0.000,0.015,0.003,0.001,0.000,2.0,300.0,0.0,300.0,4.0
4,130382.0,75846.746917,172361.851828,1.0,90.0,2.0,2.0,35.0,7.0,1.0,...,0.005,0.065,0.010,0.017,0.003,2.0,1110.0,1110.0,0.0,1.0


In [677]:
total_data = pd.merge(body, day_2, on='id', how='left')

total_data.shape

(8860, 86)

In [678]:
total_data = pd.merge(total_data, total_nutrients_1, on='id', how='left')
total_data.shape

(8860, 253)

In [679]:
total_data = pd.merge(total_data, adult_demo, on='id', how='left')
total_data.shape

(8860, 279)

In [680]:
total_data = pd.merge(total_data, sleep, on='id', how='left')
total_data.shape

(8860, 285)

In [681]:
# total_data = pd.merge(total_data, income, on='id', how='left')

# total_data.shape

In [682]:
# total_data = pd.merge(total_data, insulin, on='id', how='left')

# total_data.shape

In [683]:
# total_data = pd.merge(total_data, depression, on='id', how='left')

# total_data.shape

In [684]:
total_data = pd.merge(physical_activity, total_data, on='id', how='left')

total_data.shape

(8104, 294)

In [685]:
total_data = pd.merge(total_data, alcohol, on='id', how='left')

total_data.shape

(8104, 302)

In [686]:
total_data = pd.merge(total_data, income, on='id', how='left')

total_data.shape

(8104, 305)

In [687]:
all_cols = total_data.columns.tolist()

all_cols

['id',
 'Frequency of moderate LTPA',
 'Moderate LTPA unit (day/week/month/year)',
 'Minutes moderate LTPA',
 'Frequency of vigorous LTPA',
 'Vigorous LTPA unit (day/week/month/year)',
 'Minutes vigorous LTPA',
 'Minutes sedentary activity',
 'min_mod_week',
 'min_vig_week',
 'bmi',
 'Dietary day one sample weight_x',
 'Dietary two-day sample weight_x',
 'Dietary recall status_x',
 'Interviewer ID code_x',
 'Breast-fed infant (either day)_x',
 'Number of days of intake_x',
 '# of days between intake and HH interview',
 'Intake day of the week_x',
 'Language respondent used mostly_x',
 'Main respondent for this interview_x',
 'Helped in responding for this interview_x',
 'Number of foods/beverages reported_x',
 'Salt used at table yesterday?_x',
 'Type of salt used yesterday_x',
 'Energy (kcal)_x',
 'Protein (gm)_x',
 'Carbohydrate (gm)_x',
 'Total sugars (gm)_x',
 'Dietary fiber (gm)_x',
 'Total fat (gm)_x',
 'Total saturated fatty acids (gm)_x',
 'Total monounsaturated fatty acids (gm

In [688]:
sel_cols_2 = [
 'Family monthly poverty level index',
 'Education level - Adults 20+',
 'avg_drinks_day',
 'Sleep hours - weekdays or workdays',
 'min_mod_week',
 'min_vig_week',
 'Gender',
 'Age in years at screening',
 'bmi',
 'Energy (kcal)_x',
 'Protein (gm)_x',
 'Carbohydrate (gm)_x',
 'Total sugars (gm)_x',
 'Dietary fiber (gm)_x',
 'Total fat (gm)_x',
 'Total saturated fatty acids (gm)_x',
 'Total monounsaturated fatty acids (gm)_x',
 'Total polyunsaturated fatty acids (gm)_x',
 'Cholesterol (mg)_x',
#  'Vitamin E as alpha-tocopherol (mg)_x',
#  'Added alpha-tocopherol (Vitamin E) (mg)_x',
#  'Retinol (mcg)_x',
#  'Vitamin A, RAE (mcg)_x',
#  'Alpha-carotene (mcg)_x',
#  'Beta-carotene (mcg)_x',
#  'Beta-cryptoxanthin (mcg)_x',
#  'Lycopene (mcg)_x',
#  'Lutein + zeaxanthin (mcg)_x',
#  'Thiamin (Vitamin B1) (mg)_x',
#  'Riboflavin (Vitamin B2) (mg)_x',
#  'Niacin (mg)_x',
#  'Vitamin B6 (mg)_x',
#  'Total folate (mcg)_x',
#  'Folic acid (mcg)_x',
#  'Food folate (mcg)_x',
#  'Folate, DFE (mcg)_x',
#  'Total choline (mg)_x',
#  'Vitamin B12 (mcg)_x',
#  'Added vitamin B12 (mcg)_x',
#  'Vitamin C (mg)_x',
#  'Vitamin D (D2 + D3) (mcg)_x',
#  'Vitamin K (mcg)_x',
#  'Calcium (mg)_x',
#  'Phosphorus (mg)_x',
#  'Magnesium (mg)_x',
#  'Iron (mg)_x',
#  'Zinc (mg)_x',
#  'Copper (mg)_x',
 'Sodium (mg)_x',
 'Potassium (mg)_x',
#  'Selenium (mcg)_x',
 'Caffeine (mg)_x',
#  'Theobromine (mg)_x',
#  'Alcohol (gm)_x',
#  'Moisture (gm)_x',
#  'SFA 4:0 (Butanoic) (gm)_x',
#  'SFA 6:0 (Hexanoic) (gm)_x',
#  'SFA 8:0 (Octanoic) (gm)_x',
#  'SFA 10:0 (Decanoic) (gm)_x',
#  'SFA 12:0 (Dodecanoic) (gm)_x',
#  'SFA 14:0 (Tetradecanoic) (gm)_x',
#  'SFA 16:0 (Hexadecanoic) (gm)_x',
#  'SFA 18:0 (Octadecanoic) (gm)_x',
#  'MFA 16:1 (Hexadecenoic) (gm)_x',
#  'MFA 18:1 (Octadecenoic) (gm)_x',
#  'MFA 20:1 (Eicosenoic) (gm)_x',
#  'MFA 22:1 (Docosenoic) (gm)_x',
#  'PFA 18:2 (Octadecadienoic) (gm)_x',
#  'PFA 18:3 (Octadecatrienoic) (gm)_x',
#  'PFA 18:4 (Octadecatetraenoic) (gm)_x',
#  'PFA 20:4 (Eicosatetraenoic) (gm)_x',
#  'PFA 20:5 (Eicosapentaenoic) (gm)_x',
#  'PFA 22:5 (Docosapentaenoic) (gm)_x',
#  'PFA 22:6 (Docosahexaenoic) (gm)_x',
 'Total plain water drank yesterday (gm)_x',
 'Energy (kcal)_y',
 'Protein (gm)_y',
 'Carbohydrate (gm)_y',
 'Total sugars (gm)_y',
 'Dietary fiber (gm)_y',
 'Total fat (gm)_y',
 'Total saturated fatty acids (gm)_y',
 'Total monounsaturated fatty acids (gm)_y',
 'Total polyunsaturated fatty acids (gm)_y',
 'Cholesterol (mg)_y',
#  'Vitamin E as alpha-tocopherol (mg)_y',
#  'Added alpha-tocopherol (Vitamin E) (mg)_y',
#  'Retinol (mcg)_y',
#  'Vitamin A, RAE (mcg)_y',
#  'Alpha-carotene (mcg)_y',
#  'Beta-carotene (mcg)_y',
#  'Beta-cryptoxanthin (mcg)_y',
#  'Lycopene (mcg)_y',
#  'Lutein + zeaxanthin (mcg)_y',
#  'Thiamin (Vitamin B1) (mg)_y',
#  'Riboflavin (Vitamin B2) (mg)_y',
#  'Niacin (mg)_y',
#  'Vitamin B6 (mg)_y',
#  'Total folate (mcg)_y',
#  'Folic acid (mcg)_y',
#  'Food folate (mcg)_y',
#  'Folate, DFE (mcg)_y',
#  'Total choline (mg)_y',
#  'Vitamin B12 (mcg)_y',
#  'Added vitamin B12 (mcg)_y',
#  'Vitamin C (mg)_y',
#  'Vitamin D (D2 + D3) (mcg)_y',
#  'Vitamin K (mcg)_y',
#  'Calcium (mg)_y',
#  'Phosphorus (mg)_y',
#  'Magnesium (mg)_y',
#  'Iron (mg)_y',
#  'Zinc (mg)_y',
#  'Copper (mg)_y',
 'Sodium (mg)_y',
 'Potassium (mg)_y',
#  'Selenium (mcg)_y',
 'Caffeine (mg)_y',
#  'Theobromine (mg)_y',
#  'Alcohol (gm)_y',
#  'Moisture (gm)_y',
#  'SFA 4:0 (Butanoic) (gm)_y',
#  'SFA 6:0 (Hexanoic) (gm)_y',
#  'SFA 8:0 (Octanoic) (gm)_y',
#  'SFA 10:0 (Decanoic) (gm)_y',
#  'SFA 12:0 (Dodecanoic) (gm)_y',
#  'SFA 14:0 (Tetradecanoic) (gm)_y',
#  'SFA 16:0 (Hexadecanoic) (gm)_y',
#  'SFA 18:0 (Octadecanoic) (gm)_y',
#  'MFA 16:1 (Hexadecenoic) (gm)_y',
#  'MFA 18:1 (Octadecenoic) (gm)_y',
#  'MFA 20:1 (Eicosenoic) (gm)_y',
#  'MFA 22:1 (Docosenoic) (gm)_y',
#  'PFA 18:2 (Octadecadienoic) (gm)_y',
#  'PFA 18:3 (Octadecatrienoic) (gm)_y',
#  'PFA 18:4 (Octadecatetraenoic) (gm)_y',
#  'PFA 20:4 (Eicosatetraenoic) (gm)_y',
#  'PFA 20:5 (Eicosapentaenoic) (gm)_y',
#  'PFA 22:5 (Docosapentaenoic) (gm)_y',
#  'PFA 22:6 (Docosahexaenoic) (gm)_y',
 'Total plain water drank yesterday (gm)_y'
 ]

In [689]:
total_data = total_data[sel_cols_2]

total_data = total_data.fillna(0)
total_data.shape


(8104, 37)

In [690]:
# Linear Regression

y = total_data['bmi'].astype(int)
X = total_data.drop(['bmi'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8 )

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled) 

print(f"Mean squared error: {mean_squared_error(y_test, y_pred)}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred)}")

Mean squared error: 57.74063283127406
Coefficient of determination: 0.6978394164705416


In [691]:
# Initializing and training Lasso model

lasso_model = Lasso(alpha=0.1, max_iter=20000)
lasso_model.fit(X_train_scaled, y_train)

Lasso(alpha=0.1, max_iter=20000)

In [692]:
y_pred = lasso_model.predict(X_test_scaled)

print(f"Mean squared error: {mean_squared_error(y_test, y_pred)}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred)}")

Mean squared error: 58.07285100576112
Coefficient of determination: 0.696100896600913


In [693]:
nonzero_indices = np.nonzero(lasso_model.coef_)
nonzero_indices

cols = []
for idx in nonzero_indices[0]:
    cols.append(X_train.columns.tolist()[idx])

print(cols)

['Family monthly poverty level index', 'Education level - Adults 20+', 'avg_drinks_day', 'Sleep hours - weekdays or workdays', 'Gender', 'Age in years at screening', 'Total sugars (gm)_x', 'Dietary fiber (gm)_x', 'Total saturated fatty acids (gm)_x', 'Cholesterol (mg)_x', 'Sodium (mg)_x', 'Potassium (mg)_x', 'Total plain water drank yesterday (gm)_x', 'Carbohydrate (gm)_y', 'Total sugars (gm)_y', 'Dietary fiber (gm)_y', 'Total fat (gm)_y', 'Sodium (mg)_y', 'Potassium (mg)_y', 'Total plain water drank yesterday (gm)_y']


In [694]:
X.shape

(8104, 36)